In [ ]:
%reload_ext autoreload
%autoreload 2

import acr
import kdephys as kde
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import plotly.express as px

import warnings
warnings.filterwarnings('ignore')
import os
import scipy

In [ ]:
def get_all_probe_subs_exps():
    all_probe_analyzed = []
    for f in os.listdir('/Volumes/ceph-tononi/acr_archived_subjects/dlc_home_cage_model/inference_results'):
        if f.endswith('.csv') and 'filtered' not in f and 'ACR_' in f:
            all_probe_analyzed.append(f)

    all_probe_subs_exps = []
    for f in all_probe_analyzed:
        sub_exp = f.split('DLC_')[0]
        all_probe_subs_exps.append(sub_exp)

    return all_probe_subs_exps

In [ ]:
all_probe_subs_exps = get_all_probe_subs_exps()
cdfs = {}
vels = {}
hd = {}

for f in all_probe_subs_exps:
    sub, rec = f.split('--')
    cdf = acr.dlc.load_probe_points_df(sub, rec, filtered=False)
    vel = acr.dlc.compute_full_velocity_df(cdf)
    h = acr.io.load_hypno(sub, rec, corrections=True)
    cdfs[f] = cdf
    vels[f] = vel
    hd[f] = h

# All Nodes, average, then scale

In [ ]:
sub_dfs = []
ss=['NREM', 'REM', 'Transition-to-REM', 'Transition-to-NREM', 'Transition-to-Wake']
for f in vels.keys():
    v = vels[f]
    sub, rec = f.split('--')
    spd = v.group_by('datetime').agg(pl.col('speed').mean()).sort(['datetime'])
    spd = acr.dlc.rob_z_col(spd, 'speed')
    h = hd[f]
    spd = spd.ts(h['start_time'].min(), h['end_time'].max())
    h_sleep = acr.dlc.get_hypno_percent_asleep(h, ss)
    mean = spd['speed_robust_z'].mean()
    std = spd['speed_robust_z'].std()
    base = mean/std
    coeff_tests = [2, 1.5, 1, 0.5, 0.3, 0.2, 0.1, 0, -0.1, -0.2,-0.3, -0.4, -0.45, -0.5, -0.6, -0.7, -0.8, -0.9, -1, -1.1, -1.2, -1.3, -1.4, -1.5, -1.6, -1.7, -1.8, -1.9, -2]
    thresh_sleeps = []
    for c in coeff_tests:
        thresh_sleep = base*c
        thresh_sleep_pct = acr.dlc.threshold_sleep_on_diff_df(spd, thresh_sleep, col='speed_robust_z')
        thresh_sleeps.append(thresh_sleep_pct)
    sub_df = pd.DataFrame({'subject': sub, 'rec': rec, 'thresh_sleep': thresh_sleeps, 'h_sleep': h_sleep, 'coeffs': coeff_tests, 'mean': mean, 'std': std})
    sub_dfs.append(sub_df)

sdf = pd.concat(sub_dfs)
sdf['diff'] = sdf['thresh_sleep'] - sdf['h_sleep']

In [ ]:
f, ax = plt.subplots(figsize=(20, 10))
sns.boxplot(data=sdf, x='coeffs', y='diff')
ax.axhline(0, color='red', linestyle='--')

In [ ]:
durs = []
for hk in hd.keys():
    hypno = hd[hk]
    dur = hypno['duration'].sum().total_seconds()
    durs.append(dur)

In [ ]:
#seconds
np.sum(durs)

In [ ]:
# hours
1421023.46404/3600

In [ ]:
# days
1421023.46404/3600/24

In [ ]:
df = sdf.loc[sdf['coeffs']==-1]
thresholded_total_sleep = df['thresh_sleep'].values
hypno_total_sleep = df['h_sleep'].values
diffs = thresholded_total_sleep - hypno_total_sleep
df['diffs'] = diffs

In [ ]:
plt.rcdefaults()
acr.plots.supl()

In [ ]:
f, ax = acr.plots.gen_paired_boxplot(hypno_total_sleep, thresholded_total_sleep, colors=['darkorange', 'dodgerblue'], mean_color='black', alphas=[1, 1], widths=0.05)
ax.set_xticks([])
fig_name = f'{acr.utils.PAPER_FIGURE_ROOT}/nor_behavior/act_validation_hypno_vs_thresh'
f.savefig(f'{fig_name}.png', transparent=True, dpi=600, bbox_inches='tight')

In [ ]:
np.mean(diffs)*100

In [ ]:
f, ax = plt.subplots(figsize=(3, 4))
acr.plots.add_boxplot(ax, diffs, color='darkorange', mean_color='dodgerblue', alpha=1, widths=0.08, positions=[0.45])
acr.plots.add_data_points(ax, diffs, color='darkorange', alpha=0.8, x_pos=0.52)
ax.set_xlim(0.35, 0.6)
ax.set_xticks([0.45])
ax.set_xticklabels(['Difference'])

fig_name = f'{acr.utils.PAPER_FIGURE_ROOT}/nor_behavior/act_validation_hypno_vs_thresh--DIFFERENCE'
f.savefig(f'{fig_name}.png', transparent=True, dpi=600, bbox_inches='tight')

In [ ]:
ss=['NREM', 'REM', 'Transition-to-REM', 'Transition-to-NREM', 'Transition-to-Wake']
fkey = 'ACR_35--swi2-bl'
for fkey in vels.keys():
    v = vels[fkey]
    spd = v.group_by('datetime').agg(pl.col('speed').mean()).sort(['datetime'])
    spd = acr.dlc.rob_z_col(spd, 'speed')
    h = hd[fkey]
    spd = spd.ts(h['start_time'].min(), h['end_time'].max())
    h_sleep = acr.dlc.get_hypno_percent_asleep(h, ss)
    mean = spd['speed_robust_z'].mean()
    std = spd['speed_robust_z'].std()
    base = mean/std
    spd = spd.with_columns(state=pl.lit('Wake'))
    spd = spd.with_columns(pl.when(pl.col('speed_robust_z') < base).then(pl.lit('NREM')).otherwise(pl.col('state')).alias('state'))
    spd_df = spd.to_pandas()
    # 1.  Detect where the state changes ------------------------------------------
    transition = spd_df['state'] != spd_df['state'].shift(fill_value=spd_df['state'].iloc[0])

    # 2.  Give every contiguous block a unique id ---------------------------------
    segment_id = transition.cumsum()

    # 3.  Aggregate per block ------------------------------------------------------
    hypno = (
        spd_df
        .assign(segment=segment_id)               # add the id just created
        .groupby('segment')
        .agg(
            start_time = ('datetime', 'first'),       # <-- replace 'time' with spd_df.index if needed
            end_time   = ('datetime', 'last'),
            state      = ('state', 'first')
        )
        .reset_index(drop=True)
    )

    # 4.  Compute duration ---------------------------------------------------------
    hypno['duration'] = hypno['end_time'] - hypno['start_time']

    # 5.  Optional: reorder columns / inspect --------------------------------------
    hypno = hypno[['start_time', 'end_time', 'state', 'duration']]

    #Plot the real hypnogram
    ls = acr.hypnogram_utils.get_light_schedule(h)
    f, ax = plt.subplots(figsize=(24, 8))
    ax = sns.lineplot(data=spd, x='datetime', y='speed_robust_z', ax=ax)
    h.loc[h['state'].isin(ss), 'state'] = 'NREM'
    ax = kde.plot.main.shade_hypno_for_me(h, ax, alpha=0.3)
    ax = kde.plot.main.add_light_schedule(ls, ax)
    ax.set_title(f'{fkey}')
    plt.show()

    # Plot the hypnogram from the model
    f, ax = plt.subplots(figsize=(24, 8))
    ax = sns.lineplot(data=spd, x='datetime', y='speed_robust_z', ax=ax)
    ax = kde.plot.main.shade_hypno_for_me(hypno, ax, alpha=0.3)
    ax = kde.plot.main.add_light_schedule(ls, ax)
    ax.set_title(f'{fkey}')
    plt.show()jkk

In [ ]:
ss=['NREM', 'REM', 'Transition-to-REM', 'Transition-to-NREM', 'Transition-to-Wake']
fkey = 'ACR_35--swisin2-bl'

v = vels[fkey]
spd = v.group_by('datetime').agg(pl.col('speed').mean()).sort(['datetime'])
spd = acr.dlc.rob_z_col(spd, 'speed')
h = hd[fkey]
spd = spd.ts(h['start_time'].min(), h['end_time'].max())
h_sleep = acr.dlc.get_hypno_percent_asleep(h, ss)
mean = spd['speed_robust_z'].mean()
std = spd['speed_robust_z'].std()
base = mean/std
spd = spd.with_columns(state=pl.lit('Wake'))
spd = spd.with_columns(pl.when(pl.col('speed_robust_z') < base).then(pl.lit('NREM')).otherwise(pl.col('state')).alias('state'))
spd_df = spd.to_pandas()
# 1.  Detect where the state changes ------------------------------------------
transition = spd_df['state'] != spd_df['state'].shift(fill_value=spd_df['state'].iloc[0])

# 2.  Give every contiguous block a unique id ---------------------------------
segment_id = transition.cumsum()

# 3.  Aggregate per block ------------------------------------------------------
hypno = (
    spd_df
    .assign(segment=segment_id)               # add the id just created
    .groupby('segment')
    .agg(
        start_time = ('datetime', 'first'),       # <-- replace 'time' with spd_df.index if needed
        end_time   = ('datetime', 'last'),
        state      = ('state', 'first')
    )
    .reset_index(drop=True)
)

# 4.  Compute duration ---------------------------------------------------------
hypno['duration'] = hypno['end_time'] - hypno['start_time']

# 5.  Optional: reorder columns / inspect --------------------------------------
hypno = hypno[['start_time', 'end_time', 'state', 'duration']]


ls = acr.hypnogram_utils.get_light_schedule(h)
f, ax = plt.subplots(figsize=(24, 8))
ax = sns.lineplot(data=spd, x='datetime', y='speed_robust_z', ax=ax)
ax.set_xlim(spd['datetime'].min(), spd['datetime'].max())
fig_name = f'{acr.utils.PAPER_FIGURE_ROOT}/nor_behavior/speed_robustZ_{fkey}'
f.savefig(f'{fig_name}.png', transparent=True, dpi=600, bbox_inches='tight')


#Plot the real hypnogram
f, ax = plt.subplots(figsize=(24, 8))
#ax = sns.lineplot(data=spd, x='datetime', y='speed_robust_z', ax=ax)
h.loc[h['state'].isin(ss), 'state'] = 'NREM'
ax.set_xlim(spd['datetime'].min(), spd['datetime'].max())
ax = kde.plot.main.shade_hypno_for_me(h, ax, alpha=0.25)
ax = kde.plot.main.add_light_schedule(ls, ax)
fig_name = f'{acr.utils.PAPER_FIGURE_ROOT}/nor_behavior/hypnogram_{fkey}__REAL'
f.savefig(f'{fig_name}.png', transparent=True, dpi=600, bbox_inches='tight')

# Plot the hypnogram from the model
f, ax = plt.subplots(figsize=(24, 8))
#ax = sns.lineplot(data=spd, x='datetime', y='speed_robust_z', ax=ax)
ax.set_xlim(spd['datetime'].min(), spd['datetime'].max())
ax = kde.plot.main.shade_hypno_for_me(hypno, ax, alpha=0.3)
ax = kde.plot.main.add_light_schedule(ls, ax)
fig_name = f'{acr.utils.PAPER_FIGURE_ROOT}/nor_behavior/hypnogram_{fkey}__ACTIGRAPHY'
f.savefig(f'{fig_name}.png', transparent=True, dpi=600, bbox_inches='tight')